In [1]:
import os
import keras
import six
import math
import numpy as np

from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler

from matplotlib import pyplot
import csv
from keras.callbacks import CSVLogger
from keras.models import load_model


Using TensorFlow backend.


In [2]:
#parameters
img_size =224
batch_size = 1
n_classes=4

epochCount = 50
augmentationScale = 1
learningRate = 0.0001

In [3]:
# Data is not splitted as very less data was available
train_path = 'computer_vision'
val_path = 'computer_vision'

resultPath = 'result_out/'
if not os.path.exists(resultPath):
    os.makedirs(resultPath)

In [5]:
model = Sequential()
model.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(16, kernel_size=3, activation='relu'))
model.add(Conv2D(16, kernel_size=3, activation='relu'))

model.add(Flatten())

model.add(Dense(64,))
model.add(Activation('relu'))
model.add(Dense(32,))
model.add(Activation('relu'))

model.add(Dense(n_classes,))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 128)     3584      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 218, 218, 32)      18464     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 216, 216, 16)      4624      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 214, 214, 16)      2320      
_________________________________________________________________
flatten_1 (Flatten)          (None, 732736)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [7]:
# Load data
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [8]:
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_path,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

Found 88 images belonging to 4 classes.
Found 88 images belonging to 4 classes.


In [9]:
print (train_generator.class_indices)

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}


In [10]:
#Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=learningRate, decay=1e-2/epochCount),
              metrics=['acc'])

In [11]:
# Early Checkpoints and Intermediate model saving
filePath=resultPath+"cv-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filePath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
es = EarlyStopping(monitor='val_acc', mode='max', patience=10)
csv_logger = CSVLogger(resultPath+'training_log.csv', append=True, separator=',')
callbacks_list = [checkpoint,es,csv_logger]

In [12]:
#Training
history = model.fit_generator(
      train_generator,
      epochs=epochCount,
      validation_data=validation_generator,
      steps_per_epoch=(train_generator.samples*augmentationScale)//train_generator.batch_size,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1, 
      callbacks= callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
88/88 [==============================] - 152s 2s/step - loss: 1.4495 - acc: 0.3182 - val_loss: 0.6831 - val_acc: 0.2500

Epoch 00001: val_acc improved from -inf to 0.25000, saving model to result_out/cv-01-0.25.hdf5
Epoch 2/50
88/88 [==============================] - 157s 2s/step - loss: 1.3976 - acc: 0.2614 - val_loss: 1.3105 - val_acc: 0.2500

Epoch 00002: val_acc did not improve from 0.25000
Epoch 3/50
88/88 [==============================] - 134s 2s/step - loss: 1.2896 - acc: 0.2955 - val_loss: 1.4123 - val_acc: 0.2500

Epoch 00003: val_acc did not improve from 0.25000
Epoch 4/50
88/88 [==============================] - 142s 2s/step - loss: 1.2104 - acc: 0.3295 - val_loss: 1.2598 - val_acc: 0.5455

Epoch 00004: val_acc improved from 0.25000 to 0.54545, saving model to result_out/cv-04-0.55.hdf5
Epoch 5/50
88/88 [==============================] - 164s 2s/step - loss: 1.1577 - acc: 0.4432 - val_loss: 1.0667 - val_acc: 0.5682


KeyboardInterrupt: 